In [1]:
import requests
from elasticsearch import Elasticsearch
import json
from elasticsearch.helpers import scan
from elasticsearch_dsl import Search,Q
from credentials import usr_pwd

In [2]:
import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from nltk.tokenize import sent_tokenize
import numpy as np
print("TF version: ", tf.__version__)
print("Hub version: ", hub.__version__)

/home/pavan_rs/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


TF version:  2.9.1
Hub version:  0.12.0


### update ES document mappings with dense vectors

In [3]:
hosts = ["10.1.8.6:9200"]
uname, pwd =  usr_pwd()

In [4]:
es = Elasticsearch(hosts,
                  timeout = 6000,
                  use_ssl = True,
                  verify_certs = False,
                  ssl_show_warn = False,
                  raise_on_error = False,
                  http_auth = (uname,pwd)
                  )

In [5]:
es.ping()

True

In [6]:
#to create mapping for dense vector
# create_query = {
#     "mappings": {
#         "properties": {
#             "content_vector": {
#                 "type": "dense_vector",
#                 "dims": 384
#             }
#         }
#     }
# }
# es.indices.create(index="test_cantire-2021-10", body=create_query)

In [6]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L12-v2')

In [7]:
def clean_mail(mail_):
    email_content = mail_['_source']['content']

    email_content = email_content.split('From')[0].split('__')[0]
    if not (email_content or email_content == None):

        email_content = (' '.join(''.join(email_content.split('\n')).split('\r')).lower())
        email_content = ' '.join(email_content.split('\t'))
        
    return email_content

In [8]:
def get_elastic_results(es):
    
    res = es.search(index = "test_cantire-2021-10",body = {
    "size": 500,
    "query": {
    "bool": {
#     "filter":{
    "must_not": [
    {
    "exists": {
    "field": "doc_vectors2"
    }
    }
#     },
    ],
    "must": [
    {
    "match_phrase": {
    "fileType": "Message/Email File"
    }
    }
    ]
    }
    }
    })
    
    return res

In [9]:
g = get_elastic_results(es)
g["hits"]["hits"][0]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


{'_index': 'test_cantire-2021-10',
 '_type': '_doc',
 '_id': 'QB1PR01MB2467929847E110A5E769AB3AE0B99@QB1PR01MB2467.CANPRD01.PROD.OUTLOOK.COM',
 '_score': 0.057642303,
 '_source': {'content': "I know but we need to do enhancements on them still even if the basic look.\nTy\nA.\n\nFrom: Amanda O'Murtagh <Amanda.OMurtagh@fglsports.com>\nSent: October 15, 2021 1:36 PM\nTo: Adrienne Desrosiers <Adrienne.Desrosiers@fglsports.com>\nSubject: RE: Elevated windows\n\nMega windows have been cancelled\n\n\nAmanda O'Murtagh | Manager, Visual Assets | SportChek | O 403.592.8290 EXT. 80026 | amanda.omurtagh@fglsports.com<mailto:amanda.omurtagh@fglsports.com>\nCONFIDENTIALITY : This email (including attachments, if any) is confidential. If you are not the intended recipient, please notify the sender immediately and delete the message.\n\nFrom: Adrienne Desrosiers <Adrienne.Desrosiers@fglsports.com<mailto:Adrienne.Desrosiers@fglsports.com>>\nSent: October 15, 2021 1:29 PM\nTo: Amanda O'Murtagh <Amanda.O

In [51]:
from tqdm import tqdm
def update_es_vector(es, es_res):
    all_documents = es_res['hits']['hits']
    print(len(all_documents))
    if len(all_documents) > 0:
        for doc in tqdm(all_documents):
            email_content = doc['_source']['content']
            email_content = email_content.split('From')[0].split('__')[0]
            if email_content != None:
                email_content = (' '.join(''.join(email_content.split('\n')).split('\r')).lower())
                email_content = ' '.join(email_content.split('\t'))
                doc_id = doc['_id']
                _docum_ = "https://search.discoverdollar.com:9200/test_cantire-2021-10/"+ doc_id
                sent_embedding = model.encode([email_content])
                doc_vecs_ = [{'vector':list(np.array(sent_embedding)[0])}]
#                 doc_vecs_ = list(np.array(sent_embedding)[0])
#                 doc_vecs_ = [{'vector' : np.array(emb)} for emb in sent_embedding]
                try:
                    es.update(index = 'test_cantire-2021-10', doc_type = "_doc", id=doc_id, body={"doc":{"doc_vectors":doc_vecs_}})
                except:
                    print("an exception occurred")

In [42]:
es_res = get_elastic_results(es)
update_es_vector(es, es_res)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  0%|          | 0/1592 [00:00<?, ?it/s]

1592


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: The 'body' parameter is deprecated for the 'update' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
/home/pavan_rs/.local/lib/python3.7/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: [types removal] Specifying types in document update requests is deprecated, use the endpoint /{index}/_update/{id} instead.
  warnings.warn(message, category=ElasticsearchWarning)
100%|██████████| 1592/1592 [07:03<00:00,  3.76it/s]


In [44]:
# body ={
#     "size":10000,
#     "query": {
#         "exists": {
#             "field": "doc_vectors"
#         }
#     }
# }
# sample = es.search(index = "test_cantire-2021-10",body = body)
# print(len(sample["hits"]["hits"]))

In [10]:
body = {
    "size":10000,
    "query":{
        "bool":{
            "filter":{
                "exists":{
                    "field":"doc_vectors"
                }
            },
            "must":[{"match_phrase_prefix":{"fileType":"Email"}}]
        }
    }
}
result = es.search(index = "test_cantire-2021-10",body=body)
print(len(result["hits"]["hits"]))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  


1592


In [35]:
result["hits"]["hits"][0]["_source"]["doc_vectors2"][0]["vector"]
# result["hits"]["hits"][100]["_source"]["doc_vectors2"][0]["vector"]

[-0.04599197953939438,
 0.01873469538986683,
 -0.013811674900352955,
 -0.07532259821891785,
 0.008364405483007431,
 0.019438277930021286,
 -0.005326791666448116,
 0.009895474649965763,
 -0.10058045387268066,
 0.021143073216080666,
 -0.06162552163004875,
 -0.02299843728542328,
 0.024931130930781364,
 -0.05637568607926369,
 -0.009641280397772789,
 -7.208342867670581e-05,
 0.02524825558066368,
 -0.0692765936255455,
 0.020410917699337006,
 0.07362812012434006,
 -0.04034275561571121,
 0.02923150174319744,
 -0.05194927006959915,
 -0.03625013306736946,
 0.029625019058585167,
 0.00392916752025485,
 -0.056328192353248596,
 0.07317417860031128,
 -0.05616377666592598,
 -0.061673734337091446,
 -0.07980528473854065,
 -0.006788843311369419,
 -0.007594515103846788,
 -0.01198525633662939,
 0.00024533586110919714,
 -0.02662653848528862,
 -0.01735277846455574,
 -0.0017486019060015678,
 -0.035859279334545135,
 0.06420055031776428,
 -0.08643394708633423,
 -0.061909642070531845,
 -0.018656408414244652,
 0.

In [71]:
#trying to remove the unwnated field.
# body = {
#   "script": "ctx._source.remove('doc_vectors2')",
#   "query": {
#     "bool": {
#       "must": [
#         {
#           "exists": {
#             "field": "doc_vectors2"
#           }
#         }
#       ]
#     }
#   }
# }
# es.update(index = 'test_cantire-2021-10', doc_type = "_doc", body=body)

### Build API to call queries

In [14]:
import numpy as np

In [11]:
query = ['sku cost update']
query_vector = np.array(model.encode(query))[0]
query_vector = list(query_vector)
query_vector

[0.0030349481,
 0.054662637,
 0.056761153,
 0.066454686,
 0.039675426,
 -0.08861123,
 -0.07263168,
 -0.105498485,
 0.06680776,
 0.016005365,
 0.06512831,
 -0.019066494,
 0.018115183,
 -0.11052719,
 0.0056245034,
 -0.008588165,
 0.10317355,
 -0.0015092422,
 0.008292397,
 -0.05115865,
 -0.035529304,
 -0.10808008,
 -0.0067000403,
 -0.023878237,
 0.010018918,
 -0.114284314,
 0.06308673,
 0.0022136963,
 -0.053390663,
 -0.03194434,
 0.019518843,
 0.023383442,
 -0.0054032947,
 0.0056591723,
 0.019674437,
 -0.03003768,
 0.060691275,
 0.020270972,
 -0.0006180481,
 0.0750208,
 -0.025201429,
 -0.045906678,
 -0.06763719,
 0.032013208,
 0.05516867,
 -0.108729474,
 -0.09246821,
 0.0046219947,
 -0.010634105,
 0.014926229,
 0.053069543,
 0.051393833,
 -0.053840443,
 -0.05181763,
 -0.028302388,
 0.053785585,
 0.060837615,
 0.06657198,
 -0.031444374,
 0.018026706,
 -0.042344593,
 -0.039524067,
 0.06949608,
 -0.04633256,
 -0.010284352,
 -0.10315867,
 -0.03858886,
 -0.050577715,
 -0.032008126,
 -0.0701365

In [12]:
s_body = {
    "size":10,
    "query":{
        "script_score":{
            "query":{
                "exists":{"field":{"doc_vectors"}}
            },
            "script":{
                "source":"cosineSimilarity(params.queryVector,doc['doc_vectors'][0]['vector']) + 1.0",
                "params":{"queryVector": query_vector}
            } 
        }
    }
}

In [40]:
s_body = {
  "query": {
    "nested": {
      "path": "doc_vectors",
      "score_mode": "max", 
      "query": {
        "function_score": {
          "script_score": {
            "script": {
              "source": "(1.0+cosineSimilarity(params.query_vector, 'doc_vectors.vector'))",
              "params": {"query_vector": query_vector}
            }
          }
        }
      }
    }
  }
}

In [43]:
vec_res = es.search(index="test_cantire-2021-10",body=s_body)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


SerializationError: ({'size': 10, 'query': {'script_score': {'query': {'exists': {'field': {'doc_vectors'}}}, 'script': {'source': "cosineSimilarity(params.queryVector,doc['doc_vectors'][0]['vector']) + 1.0", 'params': {'queryVector': [0.0030349481, 0.054662637, 0.056761153, 0.066454686, 0.039675426, -0.08861123, -0.07263168, -0.105498485, 0.06680776, 0.016005365, 0.06512831, -0.019066494, 0.018115183, -0.11052719, 0.0056245034, -0.008588165, 0.10317355, -0.0015092422, 0.008292397, -0.05115865, -0.035529304, -0.10808008, -0.0067000403, -0.023878237, 0.010018918, -0.114284314, 0.06308673, 0.0022136963, -0.053390663, -0.03194434, 0.019518843, 0.023383442, -0.0054032947, 0.0056591723, 0.019674437, -0.03003768, 0.060691275, 0.020270972, -0.0006180481, 0.0750208, -0.025201429, -0.045906678, -0.06763719, 0.032013208, 0.05516867, -0.108729474, -0.09246821, 0.0046219947, -0.010634105, 0.014926229, 0.053069543, 0.051393833, -0.053840443, -0.05181763, -0.028302388, 0.053785585, 0.060837615, 0.06

In [58]:
vec_res["hits"]["hits"][0]

NameError: name 'vec_res' is not defined

In [23]:
s_body = {
  "query": {
    "nested": {
      "path": "doc_vectors",
      "score_mode": "max", 
      "query": {
        "function_score": {
          "script_score": {
              "script": {
                  "source": "(1.0 + cosineSimilarity(params.query_vector, 'doc_vectors.vector'))",
                  "params": {"query_vector": query_vector}
            }
          }
        }
      }
    }
  }
}

In [24]:
result_vectors = es.search(index="test_cantire-2021-10", body=s_body, size = 10)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  """Entry point for launching an IPython kernel.


RequestError: RequestError(400, 'search_phase_execution_exception', 'failed to create query: [nested] nested object under path [doc_vectors] is not of nested type')

In [92]:
for vectored_doc in result_vectors['hits']['hits']:
    print('=============', vectored_doc['_index'], '==============')
    print(clean_mail(vectored_doc))

============= cantire-test-2020-06 ==============
Sku isn’t active yet!
Ill send once I award 😊


============= cantire-test-2020-06 ==============
Sorry about that!

RFx-2019-8102: Dundas Jafine Cost Update 11/14/19


============= cantire-test-2020-06 ==============
Hi Kathryn,
Here is the dollar difference in quote costs. Should we take it back to Metaltech and ask how much of the freight costs were backed in the original prepaid quotes or can you negotiate for lower quote cost?
Current Quote
New Quote
$ Diff
Current Landed
SKU #
Freight% of Quote
New Est. Landed
Difference in Landed
134.44
127.35
-7.09
134.44
2741532
0.136
144.67
10.23
276.1
249.01
-27.09
276.1
2741533
0.319
328.44
52.34
177.54
160.7
-16.84
177.54
2741534
0.162
186.73
9.19
26.11
22.16
-3.95
26.11
2741535
0.639
36.32
10.21

[cid:image004.png@01D63B4F.4ABBB8C0]
Alyssa Ing
Category Business Analyst - Batteries, Flashlights & Work Accessories
Canadian Tire Corporation
2180 Yonge St., Toronto, ON M4P 2V8
416.480.9780 Ex

NameError: name 'vectored_doc' is not defined

###### compare with ES

In [ ]:
uri_cmp = ["https://search.discoverdollar.com:9200/cantire-2020-06/"]

es_cmp = Elasticsearch(hosts = uri_cmp, http_auth = (uname, pwd))
query = 'sku cost update'
result_cmp = es_cmp.search(body = {
        "size": 100,
        "query": {
            "match": {
                "content": query
            }
        }
    })


for result in result_cmp['hits']['hits']:
    print("==============")
    print(clean_mail(result))

In [56]:
from flask import Flask

In [57]:
app = Flask(__name__)

@app.route('/')


def search_body(model=embed):
    if request.method == 'POST':
        query = request.form.to_dict()
        query=list(to_predict_list.values())
        query_vector = np.array(model(query))
        s_body = {
          "query": {
            "nested": {
              "path": "doc_vectors",
              "score_mode": "max", 
              "query": {
                "function_score": {
                  "script_score": {
                    "script": {
                      "source": "(1.0+cosineSimilarity(params.query_vector, 'doc_vectors.vector'))",
                      "params": {"query_vector": query_vector[0]}
                    }
                  }
                }
              }
            }
          }
        }
        

    return s_body

s_body = search_body(embed)

def get_vector_documents(es_host= es_set, es_index = index, body= s_body ):
        result_vectors = es_set.search(index="cantire-test-2020-06", body=s_body, size = 100)
        for vectored_doc in result_vectors['hits']['hits']:
            print('=============', vectored_doc['_index'], '==============')
            print(clean_mail(vectored_doc))


NameError: name 'request' is not defined

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)